In [1]:
from yolox.tracker.byte_tracker import BYTETracker
from ultralytics.utils import LOGGER
from ultralytics import YOLO
import cv2

import numpy as np
import torch

# yolo 로그 메시지 비활성화
LOGGER.disabled = True

In [2]:
# YOLOv8 모델 로드 (사전 학습된 COCO 데이터셋 사용)
model = YOLO('yolov8n.pt')  # 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt' 중 선택 가능

In [3]:
# ByteTrack 설정
tracker_args = {
    'track_thresh': 0.5,  # 객체를 새로 추가할 때 사용되는 신뢰도 임계값
    'high_thresh': 0.6,   # 탐지된 객체의 신뢰도가 high_thresh보다 크면 "고신뢰 탐지"로 간주. 낮은 수치가 id 잘 유지
    'match_thresh': 0.8,  # 추적된 객체와 새 탐지된 객체 간의 매칭 기준(IOU, Intersection-over-Union).
    'track_buffer': 50,  # 객체 추적을 중단하기 전에 대기하는 프레임 수, 높은 수치가 id 잘 유지
    'mot20': False,  # MOT20 데이터셋 호환 모드를 활성화 여부. 복잡한 군중 추적 = True
}

# 딕셔너리를 Namespace로 변환
from argparse import Namespace
tracker_args = Namespace(**tracker_args)

# # BYTETracker 클래스 내부에서 딕셔너리 처리
# class BYTETracker:
#     def __init__(self, args, frame_rate=30):
#         # 딕셔너리로 처리
#         self.det_thresh = args['track_thresh'] + 0.1
#         self.buffer_size = int(frame_rate / 30.0 * args['track_buffer'])
#         self.max_time_lost = self.buffer_size
#         # 기타 초기화 코드

tracker = BYTETracker(tracker_args)

In [12]:
# 2. 로컬 영상 파일 열기
video_path = '../../../Data/video/1-1_607-C04.mp4'
url = 'rtsp://210.99.70.120:1935/live/cctv002.stream'
url2 = 'rtsp://192.168.0.108'
cap = cv2.VideoCapture(url2)

In [13]:
# 3. 프레임별 처리
object_tracks = {}  # 객체 위치 저장용 딕셔너리
count_num = 0
TARGET_CLASS = [0] #[2, 5, 7]

frame_count = 0
skip_frames = 5  # 5프레임마다 한 번씩 처리

# 첫 번째 프레임에서 프레임 크기 초기화
ret, frame = cap.read()
if not ret:  # 첫 번째 프레임 읽기 실패 시 프로그램 종료
    print("비디오 파일을 읽을 수 없습니다.")
    cap.release()
    exit()

frame_height, frame_width = frame.shape[:2]
print("프레임size (Height x Width):", frame_height, frame_width)
mid_x = frame_width // 2 - 70 # 화면 중간 x좌표 계산
mid_y = frame_height // 2  # 화면 중간 y좌표 계산

# # 가로선 길이 조정
# line_length = 100  # 가로선 길이 (400픽셀)
# start_x = mid_x + 30  # 선 시작점 (중앙 기준 왼쪽)
# end_x = mid_x + line_length  # 선 끝점 (중앙 기준 오른쪽)

# 세로선 길이 조정
line_length = 250  # 세로선 길이 (200픽셀)
start_y = 0  # 세로선 시작점 (화면 중간 기준 위쪽)
end_y = 315    # 세로선 끝점


print("영상시작")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    if frame_count % skip_frames != 0:  # 프레임 스킵
        continue
    if frame_count % 3000 == 0:  # 5000프레임마다 스트림 재연결
        print("스트림 초기화")
        cap.release()
        cap = cv2.VideoCapture(url2)

    # YOLOv8 객체 탐지
    results = model(frame)
    detections = []

    for result in results:
        for box in result.boxes:
            # print("box", box)
            x1, y1, x2, y2 = box.xyxy[0].tolist() #map(float, box.xyxy[0].tolist())  # 좌표를 계산하는 부분
            confidence = float(box.conf[0])  # tensor -> float 변환  # 신뢰도
            class_id = int(box.cls[0])  # 클래스id
            if confidence > 0.4 and class_id in TARGET_CLASS:  # 신뢰도 필터링
                detections.append([x1, y1, x2, y2, confidence, class_id])  # [x1, y1, x2, y2, confidence, class_id]
    
    # ByteTrack 추적
    img_info = (frame_height, frame_width)  # 이미지 정보
    img_size = (frame_width, frame_height)  # 이미지 크기

    if len(detections) == 0:
        detection_array = np.empty((0, 5), dtype=float)  # 빈 배열 생성
    else:
        detection_array = np.array([d[:5] for d in detections], dtype=float)#np.array(detections, dtype=float)  # 넘파이 버전 이슈로 dtype수정
    # print(f"detections: {detections}")
    detection_tensor = torch.from_numpy(detection_array).type(torch.float32)  # numpy 배열을 PyTorch Tensor로 변환
    # print("디텍탠서:", detection_tensor)
    online_targets = tracker.update(detection_tensor, img_info, img_size)
    # print("온라인타겟:", online_targets)

    # 블러 처리 준비
    blurred_frame = cv2.GaussianBlur(frame, (31, 31), 15)  # 블러 처리된 프레임 생성
    mask = np.ones_like(frame, dtype=np.uint8) * 255  # 마스크 초기화

    # 탐지된 객체의 바운딩 박스 영역 보호
    for target in online_targets:
        x1, y1, w, h = target.tlwh
        x1, y1, x2, y2 = int(x1), int(y1), int(x1 + w), int(y1 + h)
        cv2.rectangle(mask, (x1, y1), (x2, y2), (0, 0, 0), -1)  # 객체 영역을 마스크에서 제외
    
    # 블러 처리된 배경과 원본 객체 병합
    protected_frame = cv2.bitwise_and(frame, mask)  # 객체 보호 영역
    blurred_background = cv2.bitwise_and(blurred_frame, cv2.bitwise_not(mask))  # 블러 처리된 영역
    result_frame = cv2.add(protected_frame, blurred_background)  # 병합
    
    # 추적 결과 시각화
    for target in online_targets:
        # print("target.tlwh:", target)
        x1, y1, w, h = target.tlwh  # 바운딩 박스 좌표 및 크기
        x1, y1, x2, y2 = int(x1), int(y1), int(x1 + w), int(y1 + h)  # 박스 좌표 변환

        # YOLO 모델의 입력 이미지 크기 (전처리 시 사용한 크기)
        input_width, input_height = 860, 640  # 모델 입력 크기를 가져옴
        # 이미지 크기 비율 계산
        scale_x = frame_width / input_width  # 가로 비율
        scale_y = frame_height / input_height  # 세로 비율
        # 원본 이미지 크기에 맞게 좌표 변환
        x1 = int(x1 * scale_x)
        y1 = int(y1 * scale_y)
        x2 = int(x2 * scale_x)
        y2 = int(y2 * scale_y)
        scale_w = int(w * scale_x)
        scale_h = int(h * scale_y)

        # 위치 계산
        # head_x = int(x1 + (x2 - x1) / 2)  # 머리의 x 좌표 (바운딩 박스 중앙)
        # head_y = int(y1 - (y1 - y2) / 4)  # 머리의 y 좌표 (바운딩 박스 상단)
        foot_x = int(x1 + (x2 - x1) / 2)  # x좌표 (바운딩 박스 중앙)
        foot_y = int(y2 - 5)  # y좌표 y1=최상단 y2=최하단

        # 고유 객체 ID (ByteTrack에서 track_id 사용)
        object_id = target.track_id

        # 이전 위치와 비교하여 이동 방향 계산
        if object_id in object_tracks:
            # prev_head_x, prev_head_y = object_tracks[object_id]
            prev_foot_x, prev_foot_y = object_tracks[object_id]

            # 디버깅 출력
            # print(f"객체 {object_id}: prev_head_y={prev_head_y}, head_y={head_y}, mid_y={mid_y}")

            # # 머리가 위쪽 → 아래쪽 이동
            # if start_x <= head_x <= end_x:  # 머리 중심점이 가로선 범위 내에 있는 경우
            #     if prev_head_y < mid_y and head_y >= mid_y:
            #         count_num += 1
            #         print(f"객체 {object_id}가 머리가 위쪽에서 아래쪽으로 이동. 현재 인원수: {count_num}")

            #     # 머리가 아래쪽 → 위쪽 이동
            #     elif prev_head_y > mid_y and head_y <= mid_y:
            #         count_num -= 1
            #         print(f"객체 {object_id}가 머리가 아래쪽에서 위쪽으로 이동. 현재 인원수: {count_num}")

            # # 가로선 기준 위/아래 통과
            # 발이 위쪽 → 아래쪽 이동
            # if start_x <= foot_x <= end_x:  # 기준점이 가로선 범위 내에 있는 경우            
                # if prev_foot_y < mid_y and foot_y >= mid_y:
                #     count_num += 1
                #     print(f"객체 {object_id}가 다리가 위쪽에서 아래쪽으로 이동. 현재 인원수: {count_num}")

                # # 발이 래쪽 → 위쪽 이동
                # elif prev_foot_y > mid_y and foot_y <= mid_y:
                #     count_num -= 1
                #     print(f"객체 {object_id}가 다리가 아래쪽에서 위쪽으로 이동. 현재 인원수: {count_num}")

            # 세로선 기준 왼/오른쪽 통과
            if start_y <= foot_y <= end_y:  # 기준점이 세로선 범위 내에 있는 경우
                # print("조건충족!", {object_id})
                # print(f"객체 {object_id}: prev_foot_x={prev_foot_x}, prev_foot_y={prev_foot_y}")
                # print(f"객체 {object_id}: mid_x={mid_x}, mid_y={mid_y}")
                # print(f"객체 {object_id}: foot_x={foot_x}, foot_y={foot_y}")

                if prev_foot_x < mid_x and foot_x >= mid_x:
                    count_num += 1
                    print(f"객체 {object_id}가 왼쪽에서 오른쪽으로 이동. 현재 인원수: {count_num}")

                # 발이 오른쪽 → 왼쪽 이동
                elif prev_foot_x > mid_x and foot_x <= mid_x:
                    count_num -= 1
                    print(f"객체 {object_id}가 오른쪽에서 왼쪽으로 이동. 현재 인원수: {count_num}")

        # 현재 머리 위치 저장
        # object_tracks[object_id] = (head_x, head_y)
        
        # 현재 다리 위치 저장
        object_tracks[object_id] = (foot_x, foot_y)

        # # 얼굴 영역만 블러 처리 추가 (원형 블러)
        # face_x1 = max(0, x1 + int((x2 - x1) * 0.1))  # 바운딩 박스의 10% 지점, 경계값 보정
        # face_x2 = min(frame_width, x2 - int((x2 - x1) * 0.1))  # 바운딩 박스의 90% 지점, 경계값 보정
        # face_y1 = max(0, y1)
        # face_y2 = min(frame_height, y1 + int((y2 - y1) * 0.4))  # 바운딩 박스 높이의 40% 영역, 경계값 보정

        # if face_x2 > face_x1 and face_y2 > face_y1:  # 유효한 영역인지 확인
        #     # 원형 마스크 생성
        #     face_center = ((face_x1 + face_x2) // 2, (face_y1 + face_y2) // 2)
        #     face_radius = min((face_x2 - face_x1) // 2, (face_y2 - face_y1) // 2)
        #     mask = np.zeros_like(frame[face_y1:face_y2, face_x1:face_x2], dtype=np.uint8)
        #     cv2.circle(mask, (face_center[0] - face_x1, face_center[1] - face_y1), face_radius, (255, 255, 255), -1)

        #     # 블러 처리
        #     roi = frame[face_y1:face_y2, face_x1:face_x2]
        #     if roi.size > 0:  # 영역이 유효한 경우만 처리
        #         blurred_roi = cv2.GaussianBlur(roi, (31, 31), 15)
        #         masked_blur = cv2.bitwise_and(blurred_roi, mask)
        #         inverse_mask = cv2.bitwise_not(mask)
        #         original_background = cv2.bitwise_and(roi, inverse_mask)
        #         result = cv2.add(masked_blur, original_background)

        #         # 블러 처리된 영역 적용
        #         frame[face_y1:face_y2, face_x1:face_x2] = result

        # 블러 처리 추가
        # roi = frame[y1:y2, x1:x2]  # 객체 영역(ROI: Region of Interest) 선택
        # roi = frame[face_y1:face_y2, face_x1:face_x2]  # 얼굴 영역만
        # if roi.size > 0:  # 영역이 유효한 경우에만 처리
        #     blurred = cv2.GaussianBlur(roi, (31, 31), 15)  # GaussianBlur로 블러 처리 (진하기 조정 가능: 숫자 높을수록 진함)
        #     # frame[y1:y2, x1:x2] = blurred  # 블러 처리된 영역을 원본 이미지에 덮어쓰기
        #     frame[face_y1:face_y2, face_x1:face_x2] = blurred
        
        # 바운딩 박스 그리기
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # source, 왼위좌표, 오아좌표, 선 색상, 선 두께
        cv2.putText(frame, f"ID: {object_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        # cv2.circle(frame, (head_x, head_y), 5, (0, 0, 255), -1)  # 머리 위치를 빨간 점으로 표시
        cv2.circle(frame, (foot_x, foot_y), 5, (0, 0, 255), -1)  # 머리 위치를 빨간 점으로 표시
    
    # 영상에 기준선 (가로) 그리기
    # cv2.line(frame, (0, mid_y), (frame_width, mid_y), (255, 0, 0), 1)  # (255, 0, 0)은 파란색, 두께는 1
    # cv2.line(frame, (start_x, mid_y), (end_x, mid_y), (255, 0, 0), 2)  # 파란색, 두께는 2

    # 영상에 기준선 (세로) 그리기
    cv2.line(frame, (mid_x, start_y), (mid_x, end_y), (255, 0, 0), 2)

    # 현재 카운팅 값을 화면에 표시
    cv2.putText(frame, f"Count: {count_num}", (30, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # 프레임 보여주기
    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


프레임size (Height x Width): 480 640
영상시작
객체 15가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -1
스트림 초기화
객체 23가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -2
객체 31가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -1
객체 31가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -2
객체 42가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -1
객체 42가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -2
객체 39가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -1
객체 39가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -2
객체 46가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -3
스트림 초기화
객체 50가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -2
객체 50가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -3
스트림 초기화
객체 52가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -2
객체 55가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -3
스트림 초기화
스트림 초기화
객체 58가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -4
스트림 초기화
객체 59가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -3
객체 61가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -4
스트림 초기화
스트림 초기화
스트림 초기화
객체 62가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -3
객체 65가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -2
스트림 초기화
객체 66가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -3
스트림 초기화
객체 68가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -2
스트림 초기화
객체 71가 왼쪽에서 오른쪽으로 이동. 현재 인원수: -1
스트림 초기화
객체 74가 왼쪽에서 오른쪽으로 이동. 현재 인원수: 0
객체 77가 오른쪽에서 왼쪽으로 이동. 현재 인원수: -1
스트림 초기화
객체 84가 왼쪽에서 오른쪽으로 이동. 현재 인원수: 0
객체 86가 왼쪽에서 오른쪽으로 이동. 현재 인원수: 1
객체 87가 왼쪽에서 오른쪽으로 이동. 현재 인원

In [6]:
9

9